In [1]:
# 2018310064 문학준
import torch
import torch.nn as nn
import torch.nn.functional as F

/root/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torchvision.datasets as dset
import torchvision.transforms as transforms

In [3]:
from torch.utils.data import random_split

In [4]:
# Hyperparameters
batch_size = 256
learning_rate = 0.001
num_epoch = 300

In [5]:
class LeNet_5(nn.Module):
    def __init__(self):
        super(LeNet_5,self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        self.conv3 = nn.Conv2d(16, 120, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = torch.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = torch.relu(self.conv3(x))
        x = x.view(-1, 120)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)

In [6]:
transform = transforms.Compose([transforms.Resize((32,32)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5,))])

In [7]:
download_path = './CIFAR'
dataset = dset.CIFAR10(download_path, transform=transform, train = True, download = True)
test_dataset = dset.CIFAR10(download_path, transform=transform, train = False, download = True)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
train_dataset, val_dataset = random_split(dataset, [40000, 10000])

In [9]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet_5().to(device)

In [11]:
loss_funcion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.75)

In [12]:
# train

len_val_loader = len(val_loader)
len_val_dataset = len(val_dataset)

for i in range(num_epoch):
    
    val_loss = 0.0
    val_num_corrects = 0.0
    
    
    for image, label in train_loader:
        x = image.to(device)
        y_ = label.to(device)

        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_funcion(output, y_)
        loss.backward()
        optimizer.step()
        
    scheduler.step()
    
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs = val_inputs.to(device)
            val_labels = val_labels.to(device)
            val_outputs = model(val_inputs)
            loss = loss_funcion(val_outputs, val_labels)

            _, val_preds = torch.max(val_outputs, 1)
            
            val_loss += loss.item()
            val_num_corrects += torch.sum(val_preds == val_labels.data)
            
        
    train_loss = loss.cpu().detach().numpy()
    
    val_epoch_loss = val_loss / len_val_loader
    val_epoch_acc = 100 * val_num_corrects.float() / len_val_dataset
    
    if (i % 2 == 0):
        print(f"epoch : {i}, train_loss : {train_loss:.4f}, val_loss : {val_epoch_loss:.4f}, val_acc : {val_epoch_acc:.4f} %, lr : {optimizer.param_groups[0]['lr']:.6f}")

epoch : 0, train_loss : 2.3075, val_loss : 2.1355, val_acc : 32.1500 %, lr : 0.001000
epoch : 2, train_loss : 2.2635, val_loss : 2.0522, val_acc : 40.8500 %, lr : 0.001000
epoch : 4, train_loss : 2.1424, val_loss : 2.0109, val_acc : 45.0200 %, lr : 0.001000
epoch : 6, train_loss : 2.0632, val_loss : 1.9840, val_acc : 47.5400 %, lr : 0.001000
epoch : 8, train_loss : 1.8958, val_loss : 1.9587, val_acc : 49.8800 %, lr : 0.001000
epoch : 10, train_loss : 1.8868, val_loss : 1.9348, val_acc : 52.1500 %, lr : 0.001000
epoch : 12, train_loss : 1.8412, val_loss : 1.9227, val_acc : 53.6500 %, lr : 0.001000
epoch : 14, train_loss : 1.7860, val_loss : 1.9035, val_acc : 55.1900 %, lr : 0.001000
epoch : 16, train_loss : 1.7895, val_loss : 1.8981, val_acc : 55.8400 %, lr : 0.001000
epoch : 18, train_loss : 1.8028, val_loss : 1.8906, val_acc : 56.6600 %, lr : 0.001000
epoch : 20, train_loss : 1.8194, val_loss : 1.8904, val_acc : 56.5500 %, lr : 0.001000
epoch : 22, train_loss : 1.8313, val_loss : 1.87

In [13]:
# test
correct = 0
total = 0

with torch.no_grad():
    for image, label in test_loader:
        x = image.to(device)
        y_ = label.to(device)

        output = model.forward(x)
        _,output_index = torch.max(output,1)
        total += label.size(0)
        correct += (output_index == y_).sum().float()

    print("Accuracy : {}", format(100*correct/total))

Accuracy : {} 61.48999786376953


In [14]:
path = './CIFAR/model.pt'
torch.save(model.state_dict(), path)